# Bridge Pattern

Die Bridge-Methode ist ein strukturelles Entwurfsmuster, das es uns ermöglicht, die implementierungsspezifischen Abstraktionen und die implementierungsunabhängigen Abstraktionen voneinander zu trennen, so dass sie als einzelne Entitäten entwickelt werden können.
Die Bridge-Methode wird immer als eine der besten Methoden zur Organisation der Klassenhierarchie angesehen.

<b>Probleme ohne das Bridge Pattern</b>

Das Problem ist das unser TradingBot dinge macht, die ein TradingBot nicht machen sollte, z.b das Verbinden auf eine Börse sowie das abfragen der Markt daten der Börse.

<b>Wo das Bridge Pattern hilft</b>

Die Sache die das Bridge Pattern löst, ist, wennn wir zwei verschiedene Dinge haben die variiren, z.b können wir verschiedene Börsen verwenden und wir haben verschiedene TradingBots. Die Bridge gibt uns hier die Möglichkeit, eine zusätzliche Börse oder einen zusätzlichen TradingBot hinzuzufügen, ohne irgend etwas auf der anderen Seite der Bridge tun zu müssen.
Die Bridge ist also eine art Mechanismus um zwei separate Klassenhierachien zu haben, die sich unabhängig voneinander ändern können.

Wenn wir also eine Neue Börse erstellen möchten, wollen wir nicht unseren TradingBot ändern müssen, sondern wir wollen einfach eine zusätzliche Börse hinzufügen mit der das System arbeiten kann. Wenn wir uns eine Neue Trading Strategie ausdenken, möchten wir nicht die verschiedenen Arten von Börsen die wir nutzen ändern müssen, weil das nichts mit dem Trading Algorihmus zu tun hat.

#### Ohne das Bridge Pattern

In [83]:
from typing import List
import random
from abc import ABC, abstractmethod


class ExampleTradingBot(ABC):

    @staticmethod
    def _get_float_list(start, end) -> List[float]:
        random_float_list = []
        for _ in range(0, 10):
            x = round(random.uniform(start, end), 2)
            random_float_list.append(x)
        return random_float_list

    def get_actual_price(self) -> float:
        return round(random.uniform(10000,30000), 2)
    
    def get_price_history(self) -> List[float]:
        return self._get_float_list(10000, 30000)
    
    def connect(self):
        print(f"Connecting to Crypto exchange...")

    def get_market_data(self, coin: str) -> List[float]:
        return self.get_price_history()

    @abstractmethod
    def should_buy(self, last_price:float, prices: List[float]) -> bool:
        pass

    @abstractmethod
    def should_sell(self, last_price:float, prices: List[float]) -> bool:
        pass

    def check_prices(self, coin: str):
        self.connect()
        prices = self.get_market_data(coin)
        last_price = self.get_actual_price()
        should_buy = self.should_buy(last_price, prices)
        should_sell = self.should_sell(last_price, prices)
        if should_buy:
            print(f"You should buy {coin} for {last_price}€!")
        elif should_sell:
            print(f"You should sell {coin} for {last_price}€!")
        else:
            print(f"No action needed for {coin}.")
            

class ExampleAvarageTrader(ExampleTradingBot):

    def list_average(self, l: List[float]) -> float:
        return sum(l) / len(l)
    
    def should_buy(self, last_price:float, prices: List[float]) -> bool:
        return last_price < self.list_average(prices)

    def should_sell(self, last_price:float, prices: List[float]) -> bool:
        return last_price > self.list_average(prices)


class ExampleMinMaxTrader(ExampleTradingBot):

    def should_buy(self, last_price:float, prices: List[float]) -> bool:
        return last_price <= min(prices)

    def should_sell(self, last_price:float, prices: List[float]) -> bool:
        return last_price >= max(prices)


trader = ExampleAvarageTrader()
trader.check_prices("BTC/UTC")

Connecting to Crypto exchange...
You should sell BTC/UTC for 24893.67€!


In [73]:
from typing import List
import random
from abc import ABC, abstractmethod

class Exchange(ABC):
    
    @staticmethod
    def _get_float_list(start, end) -> List[float]:
        random_float_list = []
        for _ in range(0, 10):
            x = round(random.uniform(start, end), 2)
            random_float_list.append(x)
        return random_float_list
    
    @abstractmethod
    def get_actual_price(self) -> float:
        pass
    
    @abstractmethod
    def connect(self):
        pass

    @abstractmethod
    def get_market_data(self, coin: str) -> List[float]:
        pass
    

class BinanceExchange(Exchange):
    
    def connect(self) -> None:
        print(f"Connecting to Binance...")

    def get_actual_price(self) -> float:
        return round(random.uniform(200,500), 2)

    def get_market_data(self, coin: str) -> List[float]:
        return self._get_float_list(200, 500)
    
    
class CoinbaseExchange(Exchange):
    
    def connect(self) -> None:
        print(f"Connecting to Coinbase...")

    def get_actual_price(self) -> float:
        return round(random.uniform(1000, 2000), 2)

    def get_market_data(self, coin: str) -> List[float]:
        return self._get_float_list(1000, 2000)


class TradingBot(ABC):

    def __init__(self, exchange: Exchange) -> None:
        self.exchange = exchange

    @abstractmethod
    def should_buy(self, last_price:float, prices: List[float]) -> bool:
        pass

    @abstractmethod
    def should_sell(self, last_price:float, prices: List[float]) -> bool:
        pass

    def check_prices(self, coin: str):
        self.exchange.connect()
        prices = self.exchange.get_market_data(coin)
        last_price = self.exchange.get_actual_price()
        should_buy = self.should_buy(last_price, prices)
        should_sell = self.should_sell(last_price, prices)
        if should_buy:
            print(f"You should buy {coin} for {last_price}€!")
        elif should_sell:
            print(f"You should sell {coin} for {last_price}€!")
        else:
            print(f"No action needed for {coin}.")
            

class AvarageTrader(TradingBot):

    def list_average(self, l: List[float]) -> float:
        return sum(l) / len(l)
    
    def should_buy(self, last_price:float, prices: List[float]) -> bool:
        return last_price < self.list_average(prices)

    def should_sell(self, last_price:float, prices: List[float]) -> bool:
        return last_price > self.list_average(prices)


class MinMaxTrader(TradingBot):

    def should_buy(self, last_price:float, prices: List[float]) -> bool:
        return last_price <= min(prices)

    def should_sell(self, last_price:float, prices: List[float]) -> bool:
        return last_price >= max(prices)


trader = MinMaxTrader(CoinbaseExchange())
trader.check_prices('BTC/UTC')

Connecting to Coinbase...
You should buy BTC/UTC for 1141.7€!
